In [1]:
import os
import boto3
from config import *
import base64
from urllib.parse import urlparse
import random
import string
import time
import time
from datetime import timedelta

In [2]:
print(boto3.__version__)

1.37.9


In [3]:
bedrock_agent = boto3.Session().client('bedrock-agent')

In [4]:
def generate_random_code():
    # Generate 4 random characters (uppercase letters)
    characters = ''.join(random.choices(string.ascii_uppercase, k=4))
    
    # Generate 12 digit number combining timestamp and random numbers
    timestamp = str(int(time.time()))
    random_digits = ''.join(random.choices(string.digits, k=6))
    number = (timestamp + random_digits)[-12:]  # Ensure 12 digits
    
    return characters, number

In [5]:
def s3_upload(s3_location):
    
    random_code = generate_random_code()
    documentIdentifier = str(random_code[0]) + str(random_code[1]) 
    document = { 
        'content':{
            'dataSourceType': 'CUSTOM',
            'custom':{
                'customDocumentIdentifier': {
                    'id': documentIdentifier
                },
                's3Location': {
                    'bucketOwnerAccountId': bucket_owner_account,
                    'uri': s3_location
                },
                'sourceType': 'S3_LOCATION'
            }
        }
    }
    
    documents = [document]

    response = bedrock_agent.ingest_knowledge_base_documents(
        dataSourceId = data_source_id, documents = documents, knowledgeBaseId = knowledgebase_id
    )
    return response

In [6]:
bucket_name = 'bucket_name'
s3_folder = 'path/to/the/folder/of/your/files'
s3_folder_uri = 's3://' + bucket_name + '/' + s3_folder 
print(s3uri)

NameError: name 's3uri' is not defined

In [7]:
# first we test the function
s3_uri = "s3://mainbucketrockhight5461/test/knowledge-bases/pdffiles/00627fcf4add0ded.pdf"
response = s3_upload(s3_uri)
print(response)

{'ResponseMetadata': {'RequestId': 'a92667b7-e7eb-497b-be78-05e11b57ceb3', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Sun, 09 Mar 2025 17:25:44 GMT', 'content-type': 'application/json', 'content-length': '223', 'connection': 'keep-alive', 'x-amzn-requestid': 'a92667b7-e7eb-497b-be78-05e11b57ceb3', 'x-amz-apigw-id': 'HK1LsHproAMEg0g=', 'x-amzn-trace-id': 'Root=1-67cdcf17-096030e669c82cc73c904e5f'}, 'RetryAttempts': 0}, 'documentDetails': [{'dataSourceId': 'NE9ODXI7SB', 'identifier': {'custom': {'id': 'HXVA541143545751'}, 'dataSourceType': 'CUSTOM'}, 'knowledgeBaseId': '4CI1ZBRFZL', 'status': 'STARTING', 'updatedAt': datetime.datetime(2025, 3, 9, 17, 25, 44, 545089, tzinfo=tzlocal())}]}


In [7]:
def append_to_file(filename, content):
    """
    Appends a string to a file. If the file doesn't exist, it creates it.
    
    :param filename: The name of the file to append to
    :param content: The string content to append
    """
    try:
        with open(filename, 'a') as file:
            file.write(content)
        print(f"Successfully appended content to {filename}")
    except IOError as e:
        print(f"An error occurred while appending to the file: {e}")

# Example usage:
# append_to_file('example.txt', 'This is some content to append.\n')
        

In [8]:
def traverse_s3_folder(bucket_name, folder_path):
    s3 = boto3.client('s3')
    
    # List objects in the specified folder
    paginator = s3.get_paginator('list_objects_v2')
    print(paginator)
    pages = paginator.paginate(Bucket=bucket_name, Prefix=folder_path)
    print(type(pages))
    counter = 0

    processed_files = 0
    total_time = 0
    start_time = time.time()
    
    for page in pages:
        if 'Contents' in page:
            for obj in page['Contents']:
                # Get the full S3 URI
                s3_uri = f"s3://{bucket_name}/{obj['Key']}"
                if obj['Size'] == 0:
                    continue
                
                # Call the s3_upload function and measure the time
                start_upload = time.time()
                try:
                    response = s3_upload(s3_uri)
                except:
                    print("error")    
                end_upload = time.time()
                
                upload_time = end_upload - start_upload
                total_time += upload_time
                processed_files += 1

                # Calculate average time and estimated time remaining
                avg_time = total_time / processed_files
                estimated_remaining = avg_time * (total_files - processed_files)

                if counter == 0 or counter == 5 or counter % 300 == 0:
                    print(response)
                    log = f"\rProcessed: {processed_files}/{total_files} |"
                    log +=  f"Progress: {processed_files/total_files*100:.2f}% |"
                    log += f" Avg time per file: {avg_time:.2f}s"
                    log += f"| Est. time remaining: {timedelta(seconds=int(estimated_remaining))}"
                    print(log,end='', flush=True)
                    append_to_file('log.txt',log)
                    append_to_file('log.txt', str(response))
                counter +=1


In [20]:
total_files = 83000
# Example usage
bucket_name = 'bucket_name'
folder_path = 'path/to/your/folder'

In [ ]:
traverse_s3_folder(bucket_name, folder_path)

In [9]:
# we will list all the files with https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/list_knowledge_base_documents.html
response = bedrock_agent.list_knowledge_base_documents(
    dataSourceId=data_source_id,
    knowledgeBaseId=knowledgebase_id,
    maxResults=123,
)

In [10]:
print(len(response['documentDetails']))

123


In [11]:
print(response['nextToken'])
nextToken = response['nextToken']

AQICAHjY5vHh4XCChv6ADNKwebT+wYJnogYxpqLFttdk470lNwERdBhO/Ap8R+v+R2qimRUeAAABZzCCAWMGCSqGSIb3DQEHBqCCAVQwggFQAgEAMIIBSQYJKoZIhvcNAQcBMB4GCWCGSAFlAwQBLjARBAzb9HQkoB2Ad/SPoHcCARCAggEaRnAzRK4rkWsfWoVfzheOM9PWWxrqm0mnIfaV+Z+V7uXKDEyiqdzkLBUJdDbYq1ozXJ9auonJB1L1ot7GQxWsag+oVUSEmQma5rcXyeBTceCTtxi9DXdfkyWQBPSsAuRFzkuWDdLY9VDld7vXo0ObxSahvtspCoPEXcTyG0Tpzup6tiRsZ9rCV8Z9hrZDUzhWZtQBmu8T7nIciArmFXub7Sa5HCweQvXoXbFCKFGeLjVu1IVzMXKdW7cFS0eF2wYilM/FBltrhxAwZlU3LUp5TSLNwA0Cr4aSnBdiihWzP+4Bl5qLlnH512O96GXPvJyVcREDt08V6cT4KZLli7gttukZjVQsBMDej7lTsf7t1DYt08oWvP5fPnWs


In [18]:
# we will list all the files with https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/list_knowledge_base_documents.html
def list_section(nextToken):
    response = bedrock_agent.list_knowledge_base_documents(
        dataSourceId=data_source_id,
        knowledgeBaseId=knowledgebase_id,
        maxResults=123,
        nextToken = nextToken
    )
    return response

In [15]:
def sum_successful_uploads(documentDetails):
    counter = 0
    failed = 0
    for document in documentDetails:
        if document['status'] == 'FAILED':
            failed += 1
        counter += 1
    return counter - failed

In [16]:
sm = sum_successful_uploads(response['documentDetails'])
print(sm)

118


In [19]:
docsize = 5
counter = 0
while docsize > 1:
    response = list_section(nextToken)
    if 'nextToken' not in response:
        break
    nextToken = response['nextToken']
    sm += sum_successful_uploads(response['documentDetails'])
    if counter == 0 or counter == 7 or counter == 49 or counter %100 == 0:
        print(f"sum: {sm}, counter: {counter}")
    counter +=1
print(sm)

sum: 230, counter: 0
sum: 1033, counter: 7
sum: 5928, counter: 49
sum: 11858, counter: 100
sum: 23513, counter: 200
sum: 35147, counter: 300
sum: 46832, counter: 400
sum: 58470, counter: 500
sum: 70139, counter: 600
sum: 81797, counter: 700
90057


In [ ]:
print(new_response['documentDetails'])